# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key




## Store Part I results into DataFrame

In [2]:
#import csv
city_path = 'Output_Data/cities_df.csv'
#read csv
city_data_df = pd.read_csv(city_path)

city_data_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Hobart,-42.8794,147.3294,58.12,50,75,16.11,AU,1628221700
1,Lüderitz,-26.6481,15.1594,54.05,75,33,0.96,NaN,1628221700
2,Jamestown,42.0970,-79.2353,67.59,81,20,2.75,US,1628221701
3,Bambous Virieux,-20.3428,57.7575,66.22,88,40,4.61,MU,1628221701
4,Lorengau,-2.0226,147.2712,83.32,79,18,11.16,PG,1628221701


## Humidity Heatmap

In [3]:
#configure gmaps

# Access maps with unique API key
gmaps.configure(api_key=g_key)

fig = gmaps.figure()

#Use the Lat and Lng as locations and Humidity as the weight.
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"].astype(float)



#Add Heatmap layer to map.

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

## Create new DataFrame fitting weather criteria
#### Narrow down the cities to fit weather conditions.
#### Drop any rows will null values.

In [4]:
city_data_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Hobart,-42.8794,147.3294,58.12,50,75,16.11,AU,1628221700
1,Lüderitz,-26.6481,15.1594,54.05,75,33,0.96,NaN,1628221700
2,Jamestown,42.0970,-79.2353,67.59,81,20,2.75,US,1628221701
3,Bambous Virieux,-20.3428,57.7575,66.22,88,40,4.61,MU,1628221701
4,Lorengau,-2.0226,147.2712,83.32,79,18,11.16,PG,1628221701
...,...,...,...,...,...,...,...,...,...
561,Oussouye,12.4850,-16.5469,75.38,89,96,10.22,SN,1628221858
562,Fort Collins,40.5853,-105.0844,79.41,46,1,1.01,US,1628221858
563,Mangan,27.5167,88.5333,87.08,62,100,6.38,IN,1628221859
564,Fort Nelson,58.8053,-122.7002,63.99,88,75,5.75,CA,1628221649


In [5]:
#Find where temp is between 70 and 80 degrees F
perf_temp_df = city_data_df.loc[(city_data_df["Max Temp"] > 70) &
                                         (city_data_df["Max Temp"] < 80)]

#Find where wind speed is less than 10 mph
perf_wind_df = perf_temp_df.loc[perf_temp_df["Wind Speed"] < 10]

#Find where there is zero cloudiness
perf_weather_df = perf_wind_df.loc[perf_wind_df["Cloudiness"] == 0]

perf_weather_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
135,Akşehir,38.3575,31.4164,73.40,39,0,0.60,TR,1628221739
139,Santarém,-2.4431,-54.7083,75.33,100,0,3.44,BR,1628221740
149,Behshahr,36.6923,53.5526,75.15,39,0,3.65,IR,1628221743
178,Sohag,26.5570,31.6948,79.09,16,0,1.12,EG,1628221751
275,Verkhnevilyuysk,63.4453,120.3167,76.12,26,0,7.76,RU,1628221778
329,Flinders,-34.5833,150.8552,72.18,45,0,5.01,AU,1628221793
334,Cabra,37.4725,-4.4421,76.89,37,0,1.72,ES,1628221794
348,Mogoytuy,51.2833,114.9167,75.79,45,0,7.16,RU,1628221798
397,Sangar,63.9242,127.4739,73.62,27,0,9.17,RU,1628221813
401,Kalāleh,37.3807,55.4916,78.40,65,0,0.00,IR,1628221814


### Hotel Map
##### Store into variable named hotel_df.
##### Add a "Hotel Name" column to the DataFrame.
##### Set parameters to search for hotels with 5000 meters.
##### Hit the Google Places API for each city's coordinates.
##### Store the first Hotel result into the DataFrame.
##### Plot markers on top of the heatmap.

In [6]:
hotel_df = pd.DataFrame(perf_weather_df)
hotel_df["Hotel Name"] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
135,Akşehir,38.3575,31.4164,73.40,39,0,0.60,TR,1628221739,
139,Santarém,-2.4431,-54.7083,75.33,100,0,3.44,BR,1628221740,
149,Behshahr,36.6923,53.5526,75.15,39,0,3.65,IR,1628221743,
178,Sohag,26.5570,31.6948,79.09,16,0,1.12,EG,1628221751,
275,Verkhnevilyuysk,63.4453,120.3167,76.12,26,0,7.76,RU,1628221778,
329,Flinders,-34.5833,150.8552,72.18,45,0,5.01,AU,1628221793,
334,Cabra,37.4725,-4.4421,76.89,37,0,1.72,ES,1628221794,
348,Mogoytuy,51.2833,114.9167,75.79,45,0,7.16,RU,1628221798,
397,Sangar,63.9242,127.4739,73.62,27,0,9.17,RU,1628221813,
401,Kalāleh,37.3807,55.4916,78.40,65,0,0.00,IR,1628221814,


In [7]:
#Pull hotels using Google API
#list to hold resul
hotel=[]

#Set parameters
params = {
    "radius": 5000,
    "types": "hotel",
    "key": g_key
}

# Use the lat/lng from hotel_df
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    long = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{long}"

    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotel_name = requests.get(base_url, params=params).json()
    hotel.append(hotel_name["results"][1]["name"])

In [8]:
hotel_df["Hotel Name"] = hotel
hotel_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
135,Akşehir,38.3575,31.4164,73.40,39,0,0.60,TR,1628221739,GRAND HOTEL AKŞEHİR
139,Santarém,-2.4431,-54.7083,75.33,100,0,3.44,BR,1628221740,Ecotur Lodge
149,Behshahr,36.6923,53.5526,75.15,39,0,3.65,IR,1628221743,Kayer
178,Sohag,26.5570,31.6948,79.09,16,0,1.12,EG,1628221751,Doctors Club
275,Verkhnevilyuysk,63.4453,120.3167,76.12,26,0,7.76,RU,1628221778,Rosgosstrakh
329,Flinders,-34.5833,150.8552,72.18,45,0,5.01,AU,1628221793,Shellharbour Resort & Conference Centre
334,Cabra,37.4725,-4.4421,76.89,37,0,1.72,ES,1628221794,MS Fuente Las Piedras
348,Mogoytuy,51.2833,114.9167,75.79,45,0,7.16,RU,1628221798,Buryatskaya Gosudarstvennaya Sel'skokhozyaystv...
397,Sangar,63.9242,127.4739,73.62,27,0,9.17,RU,1628221813,Sberbank
401,Kalāleh,37.3807,55.4916,78.40,65,0,0.00,IR,1628221814,فرمانداری کلاله


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)


# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))